In [84]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np 
import random

# Survivability (dying in 1 turn or not) of a baseline fighter based on their position from enemy end of turn.
Hypothesis, as per general community knowlege regarding system, when the enemy is dangerous, ensuring they only gets two attacks instead of 3 is high value (step or stride).

In [85]:
# helper functions to get stat given lvl
def getMaxingConSecond(lvl):
    if (lvl<5):
        return 3
    elif(lvl<15):
        return 4
    else:
        return 5
    
def fighterArmorProfBonus(lvl):
    # trained
    if(lvl<11):
        return 2 + lvl
    # expert
    elif(lvl<17):
        return 4 + lvl
    else:
        return 6 + lvl
    
# https://2e.aonprd.com/Equipment.aspx?ID=2785
def scalingPlate(lvl):
    AC = 10 + 6 + fighterArmorProfBonus(lvl)
    if(lvl<5):
        return AC
    elif(lvl<11):
        return AC +1
    elif(lvl<18):
        return AC + 2
    else:
        return AC + 3

In [86]:
# Checks for crit, use return type as a multiplier
def strikeHit(targetAC, hitResult):
    # regular hit
    if(hitResult>=targetAC+10):
        return 2
    elif(hitResult>=targetAC):
        return 1
    else:
        return 0

In [87]:
# A generic fighter's defenses scaling with lvl
class Valeros:
    def __init__(self,lvl):
        self.actions = 3
        self.reactions = 1
        self.conMod = getMaxingConSecond(lvl)
        self.hpMax = 8 + lvl * (10 + self.conMod)
        self.hp = self.hpMax
        self.AC = scalingPlate(lvl)
    
    def regainActions():
        self.actions = 3
        self.reactions = 1

    def TurnStart():
        self.regainActions()

    def CheckHit(self,attackRoll):
        return strikeHit(self.AC,attackRoll)
    
    def TakesDamage(self,damage):
        self.hp -= damage

In [88]:
testFighter = Valeros(5)

print(testFighter.hpMax)
print(testFighter.AC)

78
24


In [89]:
#https://2e.aonprd.com/Rules.aspx?ID=2874
strikeAttackBonus_Extreme = {
-1 :	10,
0 :	10,
1 :	11,
2 :	13,
3 :	14,
4 :	16,
5 :	17,
6 :	19,
7 :	20,
8 :	22,
9 :	23,
10 :	25,
11 :	27,
12 :	28,
13 :	29,
14 :	31,
15 :	32,
16 :	34,
17 :	35,
18 :	37,
19 :	38,
20 :	40,
21 :	41,
22 :	43,
23 :	44,
24 :	46
}

def extremeStrikeAttackBonus(lvl):
    useLvl = np.clip(lvl,-1,24)
    return strikeAttackBonus_Extreme[useLvl]
 


In [90]:
def d20():
    return random.randrange(1,21)

def diceRoll(sides):
    return random.randrange(1, sides+1)

In [91]:
strikeDamage_Extreme = {
    -1 : lambda: diceRoll(6) + 1,
    0 : lambda:  diceRoll(6) + 3,
    1 : lambda:  diceRoll(8) + 4,
    2 : lambda:  diceRoll(12) + 4,
    3 : lambda:  diceRoll(12) + 8,
    4 : lambda:  2*diceRoll(10) + 7,
    5 : lambda:  2*diceRoll(12) + 7,
    6 : lambda:  2*diceRoll(12) + 10,
    7 : lambda:  2*diceRoll(12) + 12,
    8 : lambda:  2*diceRoll(12) + 15,
    9 : lambda:  2*diceRoll(12) + 17,
    10 : lambda:  2*diceRoll(12) + 20,
    11 : lambda:  2*diceRoll(12) + 22,
    12 : lambda:  3*diceRoll(12) + 19,
    13 : lambda:  3*diceRoll(12) + 21,
    14 : lambda:  3*diceRoll(12) + 24,
    15 : lambda:  3*diceRoll(12) + 26,
    16 : lambda:  3*diceRoll(12) + 29,
    17 : lambda:  3*diceRoll(12) + 31,
    18 : lambda:  3*diceRoll(12) + 34,
    19 : lambda:  4*diceRoll(12) + 29,
    20 : lambda:  4*diceRoll(12) + 32,
    21 : lambda:  4*diceRoll(12) + 34,
    22 : lambda:  4*diceRoll(12) + 37,
    23 : lambda:  4*diceRoll(12) + 39,
    24 : lambda:  4*diceRoll(12) + 42,
}

def extremeStrikeDamage(lvl):
    useLvl = np.clip(lvl,-1,24)
    return strikeDamage_Extreme[useLvl]()

In [92]:
extremeStrikeDamage(15)

41

## Sim 1 :: Fighter ends turn such that extreme threat enemy melee enemy 4x lvls above fighter can only make one attack

In [94]:
died = 0

trials = 100000
for i in range(1,trials):
    testPC = Valeros(5)

    # get hit result
    hitResult = d20() + extremeStrikeAttackBonus(9)
    didItHit = testPC.CheckHit(hitResult)
    if(didItHit):
        testPC.TakesDamage(didItHit*extremeStrikeDamage(9))


    died += testPC.hp<=0

print("Fighter dies x%:")
print(died/trials)

Fighter dies x%:
0.08378


## Sim 2 :: Fighter ends turn such that extreme threat melee enemy can make 2x attacks

In [95]:
died = 0

trials = 100000
for i in range(1,trials):
    testPC = Valeros(5)

    # get hit result
    map = [0,5]
    for curMap in map:
        hitResult = d20() + extremeStrikeAttackBonus(9) - curMap
        didItHit = testPC.CheckHit(hitResult)
        if(didItHit):
            testPC.TakesDamage(didItHit*extremeStrikeDamage(9))


    died += testPC.hp<=0

print("Fighter dies x%:")
print(died/trials)

Fighter dies x%:
0.43685


Sim 3 :: Fighter ends turn within reach of extreme threat melee enemy, 3x attacks incoming (Map applies)

In [97]:
died = 0

trials = 100000
for i in range(1,trials):
    testPC = Valeros(5)

    # get hit result
    map = [0,5,10]
    for curMap in map:
        hitResult = d20() + extremeStrikeAttackBonus(9) - curMap
        didItHit = testPC.CheckHit(hitResult)
        if(didItHit):
            testPC.TakesDamage(didItHit*extremeStrikeDamage(9))


    died += testPC.hp<=0

print("Fighter dies x%:")
print(died/trials)

Fighter dies x%:
0.62156
